# Лабораторная работа 5: Деревья решений

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
from collections import Counter
from ucimlrepo import fetch_ucirepo

# Настройка отображения графиков
%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 6)

## 1. Загрузка и предварительная обработка данных

In [ ]:
print("1. ПРЕДВАРИТЕЛЬНАЯ ОБРАБОТКА ДАННЫХ")
print("=" * 50)

# Загрузка датасета
mushroom = fetch_ucirepo(id=73)

# Данные
X = mushroom.data.features
y = mushroom.data.targets

# Преобразуем целевую переменную в бинарный формат (0 - edible, 1 - poisonous)
y_binary = (y["poisonous"] == "p").astype(int)

# Обработка пропущенных значений
X = X.fillna("unknown")

print("Информация о датасете:")
print(f"Размерность признаков: {X.shape}")
print(f"Размерность целевой переменной: {y_binary.shape}")
print()

print("Проверка отсутствующих значений:")
print(X.isnull().sum().sum())
print()

print("Первые 5 строк данных:")
print(pd.concat([X.head(), y_binary.head()], axis=1))

## 2. Статистика и визуализация данных

In [ ]:
print("2. СТАТИСТИКА И ВИЗУАЛИЗАЦИЯ ДАННЫХ")
print("=" * 50)

# Базовая статистика по целевой переменной
print("Распределение классов:")
class_dist = Counter(y_binary)
for cls, count in class_dist.items():
    percentage = count / len(y_binary) * 100
    label = "poisonous" if cls == 1 else "edible"
    print(f"{label}: {count} примеров ({percentage:.1f}%)")
print()

In [ ]:
# Визуализация распределения классов
plt.figure(figsize=(8, 6))
plt.bar(["Edible", "Poisonous"], [class_dist[0], class_dist[1]], color=["green", "red"], alpha=0.7)
plt.title("Распределение классов грибов")
plt.ylabel("Количество примеров")
plt.grid(True, alpha=0.3)
plt.show()

## 3. Реализация DecisionTree

In [ ]:
print("3. РЕАЛИЗАЦИЯ DecisionTree (НЕБИНАРНОЕ ДЕРЕВО)")
print("=" * 50)


class Node:
    """Узел небинарного дерева решений"""

    def __init__(self, feature=None, results=None, children=None):
        self.feature = feature  # Признак для разделения
        self.results = results  # Распределение классов (для листа)
        self.children = children or {}  # Словарь: значение признака -> дочерний узел


class DecisionTree:
    """Небинарное дерево решений для классификации"""

    def __init__(self, max_depth=10, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None

    def calculate_gini(self, y):
        """Вычисление коэффициента Джини"""
        if len(y) == 0:
            return 0

        class_counts = Counter(y)
        gini = 1.0

        for count in class_counts.values():
            probability = count / len(y)
            gini -= probability**2

        return gini

    def split_data_multiway(self, X, y, feature):
        """
        Разделение данных по всем уникальным значениям признака.
        Возвращает словарь: значение -> (X_subset, y_subset)
        """
        splits = {}

        for i in range(len(X)):
            value = X[i][feature]
            if value not in splits:
                splits[value] = ([], [])

            splits[value][0].append(X[i])
            splits[value][1].append(y[i])

        return splits

    def find_best_split(self, X, y, features):
        """
        Поиск лучшего признака для разделения (многопутевое разделение).
        Для каждого признака создаем ветвь для каждого уникального значения.
        """
        best_gini = float("inf")
        best_feature = None
        best_splits = None

        current_gini = self.calculate_gini(y)

        for feature in features:
            # Разделяем данные по всем уникальным значениям признака
            splits = self.split_data_multiway(X, y, feature)

            # Пропускаем признаки с единственным значением
            if len(splits) <= 1:
                continue

            # Вычисляем взвешенный коэффициент Джини
            weighted_gini = 0
            for value, (X_subset, y_subset) in splits.items():
                weight = len(y_subset) / len(y)
                weighted_gini += weight * self.calculate_gini(y_subset)

            # Выбираем лучшее разделение
            if weighted_gini < best_gini:
                best_gini = weighted_gini
                best_feature = feature
                best_splits = splits

        # Если улучшение незначительное, не разделяем
        if best_gini >= current_gini - 0.001:
            return None, None

        return best_feature, best_splits

    def build_tree(self, X, y, feature s, depth=0):
        """Рекурсивное построение небинарного дерева"""

        # Базовые случаи остановки
        if len(y) < self.min_samples_split or depth >= self.max_depth or len(set(y)) == 1:
            return Node(results=Counter(y))

        # Поиск лучшего разделения
        feature, splits = self.find_best_split(X, y, features)

        if feature is None:
            return Node(results=Counter(y))

        # Создаем дочерние узлы для каждого значения признака
        children = {}
        for value, (X_subset, y_subset) in splits.items():
            children[value] = self.build_tree(X_subset, y_subset, features, depth + 1)

        return Node(feature=feature, children=children)

    def fit(self, X, y):
        """Обучение дерева"""
        # Преобразуем DataFrame в список словарей для удобства
        X_list = X.to_dict("records")
        y_list = y.tolist()

        features = list(X.columns)

        self.root = self.build_tree(X_list, y_list, features)

    def predict_single(self, x):
        """Предсказание для одного примера"""
        node = self.root

        while node.children:
            feature_value = x[node.feature]

            # Ищем соответствующую ветвь по значению признака
            if feature_value in node.children:
                node = node.children[feature_value]
            else:
                # Если значение не встречалось при обучении, возвращаем наиболее частый класс
                break

        # Возвращаем наиболее частый класс
        if node.results:
            return max(node.results.items(), key=lambda x: x[1])[0]
        return 0

    def predict(self, X):
        """Предсказание для набора данных"""
        X_list = X.to_dict("records")
        return [self.predict_single(x) for x in X_list]

    def predict_proba_single(self, x):
        """Вероятности классов для одного примера"""
        node = self.root

        while node.children:
            feature_value = x[node.feature]

            if feature_value in node.children:
                node = node.children[feature_value]
            else:
                break

        if node.results:
            total = sum(node.results.values())
            proba_1 = node.results.get(1, 0) / total
            return [1 - proba_1, proba_1]
        return [0.5, 0.5]

    def predict_proba(self, X):
        """Вероятности классов для набора данных"""
        X_list = X.to_dict("records")
        probabilities = []

        for x in X_list:
            proba = self.predict_proba_single(x)
            probabilities.append(proba[1])  # Вероятность класса 1

        return probabilities


print("Класс DecisionTree (небинарное дерево) успешно реализован")

## 4. Отбор признаков и построение модели

In [ ]:
print("4. ОТБОР ПРИЗНАКОВ И ПОСТРОЕНИЕ МОДЕЛИ")
print("=" * 50)

# Устанавливаем seed для воспроизводимости результатов
random.seed(413732)


def select_features(X, method="sqrt"):
    """
    Отбор признаков для дерева решений
    """
    n_features = X.shape[1]

    if method == "sqrt":
        n_selected = int(math.sqrt(n_features))
    else:
        n_selected = n_features

    selected_indices = random.sample(range(n_features), n_selected)
    selected_features = X.columns[selected_indices].tolist()

    return selected_features, selected_indices


def train_test_split(X, y, test_size=0.3, random_state=None):
    """Разделение на обучающую и тестовую выборки"""
    if random_state is not None:
        np.random.seed(random_state)

    n = len(X)
    test_indices = np.random.choice(n, size=int(n * test_size), replace=False)
    train_indices = np.setdiff1d(np.arange(n), test_indices)

    X_train = X.iloc[train_indices]
    X_test = X.iloc[test_indices]
    y_train = y.iloc[train_indices]
    y_test = y.iloc[test_indices]

    return X_train, X_test, y_train, y_test


# Отбор признаков
selected_features, selected_indices = select_features(X)
X_selected = X[selected_features].copy()

print(f"Отобрано {len(selected_features)} признаков из {X.shape[1]}:")
print(selected_features)
print()

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(
    X_selected,
    y_binary,
    test_size=0.3,
    random_state=42,
)

print(f"Обучающая выборка: {len(X_train)} примеров")
print(f"Тестовая выборка: {len(X_test)} примеров")
print()

# Обучение модели
tree = DecisionTree(max_depth=5, min_samples_split=10)
tree.fit(X_train, y_train)

print("Модель дерева решений успешно обучена")

## 5. Оценка модели

In [ ]:
print("5. ОЦЕНКА МОДЕЛИ")
print("=" * 50)


def calculate_metrics(y_true, y_pred):
    """Вычисление метрик без использования библиотек"""
    tp = fp = tn = fn = 0

    for true, pred in zip(y_true, y_pred):
        if true == 1 and pred == 1:
            tp += 1
        elif true == 0 and pred == 1:
            fp += 1
        elif true == 0 and pred == 0:
            tn += 1
        elif true == 1 and pred == 0:
            fn += 1

    accuracy = (tp + tn) / (tp + fp + tn + fn) if (tp + fp + tn + fn) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0

    return accuracy, precision, recall, tp, fp, tn, fn


def calculate_auc_roc(y_true, y_proba):
    # Сортируем по убыванию вероятности
    data = sorted(zip(y_proba, y_true), key=lambda x: x[0], reverse=True)

    # Инициализация
    tpr_list = [0]
    fpr_list = [0]

    tp = fp = 0
    total_p = sum(y_true)
    total_n = len(y_true) - total_p

    last_prob = None

    for prob, true in data:
        if prob != last_prob:
            tpr = tp / total_p if total_p > 0 else 0
            fpr = fp / total_n if total_n > 0 else 0
            tpr_list.append(tpr)
            fpr_list.append(fpr)
            last_prob = prob

        if true == 1:
            tp += 1
        else:
            fp += 1

    # Добавляем конечную точку
    tpr_list.append(1)
    fpr_list.append(1)

    # Вычисление AUC методом трапеций
    auc = 0
    for i in range(1, len(tpr_list)):
        auc += (fpr_list[i] - fpr_list[i - 1]) * (tpr_list[i] + tpr_list[i - 1]) / 2

    return fpr_list, tpr_list, auc


def calculate_auc_pr(y_true, y_proba):
    # Сортируем по убыванию вероятности
    data = sorted(zip(y_proba, y_true), key=lambda x: x[0], reverse=True)

    precision_list = []
    recall_list = []

    tp = fp = 0
    total_p = sum(y_true)

    for prob, true in data:
        if true == 1:
            tp += 1
        else:
            fp += 1

        precision = tp / (tp + fp) if (tp + fp) > 0 else 1
        recall = tp / total_p if total_p > 0 else 0

        precision_list.append(precision)
        recall_list.append(recall)

    # Вычисление AUC методом трапеций
    auc_pr = 0
    for i in range(1, len(precision_list)):
        auc_pr += (
            (recall_list[i] - recall_list[i - 1]) * (precision_list[i] + precision_list[i - 1]) / 2
        )

    return precision_list, recall_list, auc_pr


# Предсказания
y_pred = tree.predict(X_test)
y_proba = tree.predict_proba(X_test)

# Оценка модели
accuracy, precision, recall, tp, fp, tn, fn = calculate_metrics(y_test.values, y_pred)

print("РЕЗУЛЬТАТЫ ОЦЕНКИ:")
print("-" * 30)
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print()

print("МАТРИЦА ОШИБОК:")
print("-" * 30)
print(f"TP: {tp}, FP: {fp}")
print(f"FN: {fn}, TN: {tn}")
print()

# Визуализация матрицы ошибок
confusion_matrix = np.array([[tn, fp], [fn, tp]])

plt.figure(figsize=(6, 5))
plt.imshow(confusion_matrix, interpolation="nearest", cmap=plt.cm.Blues)
plt.title("Матрица ошибок")
plt.colorbar()

tick_marks = np.arange(2)
plt.xticks(tick_marks, ["Predicted 0", "Predicted 1"])
plt.yticks(tick_marks, ["Actual 0", "Actual 1"])

for i in range(2):
    for j in range(2):
        plt.text(
            j,
            i,
            str(confusion_matrix[i, j]),
            horizontalalignment="center",
            color="white" if confusion_matrix[i, j] > confusion_matrix.max() / 2 else "black",
        )

plt.tight_layout()
plt.show()

In [ ]:
print("КРИВЫЕ КАЧЕСТВА:")
print("-" * 30)

# Вычисление AUC-ROC
fpr, tpr, auc_roc = calculate_auc_roc(y_test.values, y_proba)

# Вычисление AUC-PR
precision_curve, recall_curve, auc_pr = calculate_auc_pr(y_test.values, y_proba)

print(f"AUC-ROC: {auc_roc:.4f}")
print(f"AUC-PR: {auc_pr:.4f}")
print()

# Построение графиков
plt.figure(figsize=(12, 5))

# ROC-кривая
plt.subplot(1, 2, 1)
plt.plot(fpr, tpr, "b-", linewidth=2, label=f"ROC curve (AUC = {auc_roc:.4f})")
plt.plot([0, 1], [0, 1], "r--", linewidth=1, label="Random classifier")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.grid(True, alpha=0.3)

# PR-кривая
plt.subplot(1, 2, 2)
plt.plot(recall_curve, precision_curve, "g-", linewidth=2, label=f"PR curve (AUC = {auc_pr:.4f})")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Выводы

In [ ]:
print("6. ВЫВОДЫ")
print("=" * 50)

# Accuracy = (TP + TN) / (TP + TN + FP + FN)
# Precision = TP / (TP + FP) — точность положительных прогнозов
# Recall = TP / (TP + FN) — полнота обнаружения положительных

print("\nОСНОВНЫЕ РЕЗУЛЬТАТЫ:")
print("-" * 30)
print(f"• Точность модели: {accuracy:.4f}")
print(f"• Precision: {precision:.4f}")
print(f"• Recall: {recall:.4f}")
print(f"• AUC-ROC: {auc_roc:.4f}")
print(f"• AUC-PR: {auc_pr:.4f}")

print("\nХАРАКТЕРИСТИКИ МОДЕЛИ:")
print("-" * 30)
print(f"• Количество признаков: {len(selected_features)}")
print(f"• Максимальная глубина дерева: {tree.max_depth}")
print(f"• Минимальное количество samples для разделения: {tree.min_samples_split}")
print(f"• Размер тестовой выборки: {len(X_test)} примеров")

print("\nРАСПРЕДЕЛЕНИЕ КЛАССОВ В ТЕСТОВОЙ ВЫБОРКЕ:")
print("-" * 30)
test_class_dist = Counter(y_test)
for cls, count in test_class_dist.items():
    percentage = count / len(y_test) * 100
    label = "poisonous" if cls == 1 else "edible"
    print(f"• {label}: {count} примеров ({percentage:.1f}%)")